# This file is used to flat the JSON data

In [121]:
import json
import pandas as pd
import numpy as np
from shapely.geometry import Point
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5
import re
import math
import ast
from pandas.io.json import json_normalize
from flatten_json import flatten
import ProjectLookUpTable 
import socket, struct
from binascii import hexlify
import datetime
import macaddress

# Read JSON data

In [122]:
def file_json_file_with_file_name(pcap_Json_file_name):
    json_data_file = open(pcap_Json_file_name, encoding='utf-8')
    file_data = json.load(json_data_file)
    json_data_file.close()
    return file_data

In [123]:
# type_map_for_json(json_record)


def IPV6_to_int(ipv6_addr):
    return int(hexlify(socket.inet_pton(socket.AF_INET6, ipv6_addr)), 16)

def IPV4_to_int(ip):
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def type_map_for_json(json_record):
    if type(json_record) != dict:
        return True
    for json_key in json_record.keys():
        if type_map_for_json(json_record[json_key]):
            if json_key in ProjectLookUpTable.wire_shark_type_lookup_map:
                json_value_type = ProjectLookUpTable.wire_shark_type_lookup_map[json_key]
                json_str_value = json_record[json_key]
                # change number string to number 
                if json_value_type in ProjectLookUpTable.wire_shark_number_type_set:
                    if '0x' in json_str_value:
                        json_record[json_key] = int(json_str_value, 0)
                    elif '.' in json_str_value:
                        json_record[json_key] = float(json_str_value)
                    else:
                        json_record[json_key] = int(json_str_value)
                elif json_value_type in ProjectLookUpTable.wire_shark_ipaddr_type_set:
                    if ':' in json_str_value:
                        json_record[json_key] = IPV6_to_int(json_str_value)
                    else:
                        json_record[json_key] = IPV4_to_int(json_str_value)
                elif json_value_type == 'Ethernet or other MAC address':
                    json_record[json_key] = int(macaddress.MAC(json_str_value))
                # elif json_value_type == 'Date and time':
                #     json_str_value = json_str_value[0:json_str_value.index('.')+6]
                #     json_record[json_key] = datetime.datetime.strptime(json_str_value, '%b %d, %Y %H:%M:%S.%f')





In [124]:
# drop use less fields by use drop_json_fields(json_record, field_name_path_list)
def json_field_drop(json_record, path_list):
    if path_list[0] not in json_record:
        return
    if(len(path_list) == 1):
        del json_record[path_list.pop(0)]
    else:
        json_record = json_record[path_list.pop(0)]
        json_field_drop(json_record, path_list)

def drop_json_fields(json_record, field_name_path_list):
    for name_path_str in field_name_path_list:
        json_field_drop(json_record, name_path_str.split('->'))


In [125]:
def start_drop_field_value_mapping(local_json_records, local_json_drop_field_path_list):
    for local_json_record in local_json_records:
        local_json_record = local_json_record['_source']['layers']
        drop_json_fields(local_json_record, local_json_drop_field_path_list)
        type_map_for_json(local_json_record)


In [126]:
def flatten_json_records(json_record_list):
    rv = []
    for record in json_record_list:
        record = record['_source']['layers']
        rv.append(flatten(record))
    return rv

In [127]:
pcap_json_file_name = "../data/2021-02-25 - TA551 (SHATHAK) BACK TO PUSHING ICEDID (BOKBOT)/only/only.json"
json_drop_field_path_list = ['frame->frame.time', 'frame->frame.time_delta_displayed', 
    'frame->frame.coloring_rule.name', 'frame->frame.coloring_rule.string',
    'ip->ip.addr', 'ip->ip.host', 'tcp->tcp.port','http->http.request.line',
    'tcp->tcp.payload', 
    'ip->ip.src_host', 'ip->ip.dst_host', 'http->http.cookie_tree',  
    'tls->tls.record->tls.handshake->tls.handshake.random',
    'tls->tls.record->tls.handshake->tls.handshake.random_tree',
    'tls->tls.record->tls.handshake.ciphersuites',
]

json_record_list = file_json_file_with_file_name(pcap_json_file_name)
start_drop_field_value_mapping(json_record_list,json_drop_field_path_list)
flatten_json_records = flatten_json_records(json_record_list)

flatten_json_records

[{'frame_frame.encap_type': 1,
  'frame_frame.offset_shift': 0.0,
  'frame_frame.time_epoch': 1614277766.334368,
  'frame_frame.time_delta': 0.0,
  'frame_frame.time_relative': 0.0,
  'frame_frame.number': 1,
  'frame_frame.len': 292,
  'frame_frame.cap_len': 292,
  'frame_frame.marked': 0,
  'frame_frame.ignored': 0,
  'frame_frame.protocols': 'eth:ethertype:ip:tcp:http',
  'eth_eth.dst': 36168636208113,
  'eth_eth.dst_tree_eth.dst_resolved': 'Netgear_b6:93:f1',
  'eth_eth.dst_tree_eth.dst.oui': 2155818,
  'eth_eth.dst_tree_eth.dst.oui_resolved': 'Netgear',
  'eth_eth.dst_tree_eth.addr': 36168636208113,
  'eth_eth.dst_tree_eth.addr_resolved': 'Netgear_b6:93:f1',
  'eth_eth.dst_tree_eth.addr.oui': 2155818,
  'eth_eth.dst_tree_eth.addr.oui_resolved': 'Netgear',
  'eth_eth.dst_tree_eth.dst.lg': 0,
  'eth_eth.dst_tree_eth.lg': 0,
  'eth_eth.dst_tree_eth.dst.ig': 0,
  'eth_eth.dst_tree_eth.ig': 0,
  'eth_eth.src': 34395146158,
  'eth_eth.src_tree_eth.src_resolved': 'HewlettP_1c:47:ae',
  '

# TODO: map string to it matching int code, cookie spliter

# Free and testing space

In [72]:
wire_shark_type = set()
for key,val in ProjectLookUpTable.wire_shark_type_lookup_map.items():
    wire_shark_type.add(val)
wire_shark_type

{'ASN.1 object identifier',
 'Boolean',
 'Byte sequence',
 'Character string',
 'Date and time',
 'Ethernet or other MAC address',
 'Floating point (double-precision)',
 'Frame number',
 'IPv4 address',
 'IPv6 address',
 'Label',
 'Signed integer (1 byte)',
 'Signed integer (2 bytes)',
 'Signed integer (4 bytes)',
 'Signed integer (8 bytes)',
 'Time offset',
 'Unsigned integer (1 byte)',
 'Unsigned integer (2 bytes)',
 'Unsigned integer (3 bytes)',
 'Unsigned integer (4 bytes)',
 'Unsigned integer (8 bytes)',
 'Unsigned integer, 2 bytes',
 'Unsigned integer, 4 bytes'}

In [ ]:

wire_shark_type_lookup_map = {}
wire_shark_description_lookup_map = {}
for i in range(len(field_list)):
    wire_shark_type_lookup_map[field_list[i]] = type_list[i]
    wire_shark_description_lookup_map[field_list[i]] = description_list[i]
